In [103]:
import pandas as pd
import numpy as np
import os
from tqdm.autonotebook import tqdm
import datetime as dt
import requests
import getpass
import time
import json
from bs4 import BeautifulSoup
import zipfile
import glob

In [48]:
#CAMPAIGN & LOGIN INFO

print("CAMPAIGN & LOGIN INFO")
print("")

#login
USERNAME = input("Enter myElefant LDAP username:")
PASSWORD = getpass.getpass("Enter myElefant LDAP password:")

#campaign account
facebook_page_name = input("Please give the name of the Facebook page (use exact spelling used in botfbm): ")
facebook_page_id = input("Please give the id of the Facebook page : ")

#campaign dates
scenario_start = input("Please give a scenario start date (date format YYYY-MM-DD): ")
scenario_start = dt.datetime.strptime(scenario_start, "%Y-%m-%d").date()
scenario_end = input("Please give a scenario end date (date format YYYY-MM-DD): ")
scenario_end = dt.datetime.strptime(scenario_end, "%Y-%m-%d").date()

#list of shoots
shoots = input("Enter list of Campaign Ids found in https://botfbm.myelefant.com/admin/campaigns (sent) - seperated with a comma: ")
shoots = shoots.strip(" ").split(",")

CAMPAIGN & LOGIN INFO

Enter myElefant LDAP username:agreiss
Enter myElefant LDAP password:········
Please give the name of the Facebook page (use exact spelling used in botfbm): Volcom (myE)
Please give a scenario start date (date format YYYY-MM-DD): 2019-11-25
Please give a scenario end date (date format YYYY-MM-DD): 2019-11-28
Enter list of Campaign Ids found in https://botfbm.myelefant.com/admin/campaigns (sent) - seperated with a comma: 157467657422534059,157487295624274723


In [49]:
#get campaign deliverability and read infos

resps_json = []

for shoot in tqdm(shoots):
    url = "https://botfbm.myelefant.com/admin/campaigns/"+shoot+"/report"
    resp = requests.get(url, auth=(USERNAME, PASSWORD))
    resp_json = json.loads(resp.content.decode('utf8'))
    
    keys = ['campaign_id', 'campaign_short_id', 'page_id', 'messages_count', 'sending_date', 'status_code', 'submitted_messages', 
        'exclusive_submitted_messages', 'delivered_messages', 'exclusive_delivered_messages', 'read_messages', 'failed_messages', 
         'success']
    
    data = {key: resp_json[key] for key in keys}
    
    
    resps_json.append(data)
    
    time.sleep(1)
    
df_deliver_read = pd.DataFrame.from_records(resps_json)

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

In [107]:
#Function to identify and download zip file
def bot_download_file(USERNAME, PASSWORD, facebook_page_id):
    
    #clean directory by deleting all zip and csv files
    for file in glob.glob('*.csv') + glob.glob('*.zip'):
        os.remove(file)
    
    #get html of export page to get list of downloadable files
    page = requests.get("https://botfbm.myelefant.com/admin/exports/", auth=(USERNAME, PASSWORD))
    soup = BeautifulSoup(page.content, 'html.parser')
    
    #find file name to use
    now = dt.datetime.now()
    file = ""
    
    for item in tqdm(soup.find_all("td")[0:len(soup.find_all("td"))]):
        if str(now.year)+"-"+now.strftime('%m')+"-"+str(now.day - 1) in item.get_text():
            if item.get_text().endswith(".zip"):
                if facebook_page_id in item.get_text():
                    file = item.get_text()
                    break
        time.sleep(0.001)
    
    print("File found, download and unzip will begin ...")
        
    #download zip
    resp = requests.get('https://botfbm.myelefant.com/admin/static/'+file, auth=(USERNAME, PASSWORD))
    zname = os.getcwd()+"\\"+file
    zfile = open(zname, 'wb')
    zfile.write(resp.content)
    zfile.close()
    
    #unzip file
    try:
        with zipfile.ZipFile(zname, 'r') as zip_ref:
            zip_ref.extractall(os.getcwd())
    except:
        print("Error with this file : {}".format(zname))
        

In [108]:
bot_download_file(USERNAME, PASSWORD, facebook_page_id)

HBox(children=(IntProgress(value=0, max=46460), HTML(value='')))

File found, download and unzip will begin ...


In [109]:
#import full stat data files

d ={}

dtype_contacts = {'facebook_id': 'str', 'creation_time': 'str', 'last_update': 'str'}
parse_dates_contacts = ["creation_time", "last_update"]

dtype_other = {'facebook_id': 'str', 'datetime': 'str'}
parse_dates_other = ["datetime"]


for file in tqdm(os.listdir()):
    if file.endswith(".csv"):
        df_name = file.split("_")[-1].split(".")[0]
        if df_name == "contacts":
            d[df_name] =  pd.read_csv(file, sep=";", encoding="utf8", low_memory=False, dtype=dtype_contacts, parse_dates=parse_dates_contacts)
        else:
            d[df_name] =  pd.read_csv(file, sep=";", encoding="utf8", low_memory=False, dtype=dtype_other, parse_dates=parse_dates_other)
    else:
        pass

HBox(children=(IntProgress(value=0, max=8), HTML(value='')))

In [115]:
#calculate KPIs

#KPIs

#Deliverability
messages_count = df_deliver_read.messages_count.sum()
submitted = df_deliver_read.submitted_messages.sum()
delivered = df_deliver_read.delivered_messages.sum()
deliverability_rate = delivered/submitted
failed =  df_deliver_read.failed_messages.sum()


#Read
read = df_deliver_read.read_messages.sum()
read_rate = read/delivered

#Contacts
contact_date_filter_creation = (d["contacts"].creation_time >= scenario_start) & (d["contacts"].creation_time <= scenario_end)
contact_date_filter_update = (d["contacts"].last_update >= scenario_start) & (d["contacts"].last_update <= scenario_end)

optins = d["contacts"].facebook_id.nunique()
new_users = d["contacts"][contact_date_filter_creation].facebook_id.nunique()
acquisition_rate = new_users/d["contacts"][d["contacts"].creation_time <= scenario_start].facebook_id.nunique()
retention_rate = d["contacts"][(d["contacts"].creation_time <= scenario_start) & 
                               (d["contacts"].last_update >= scenario_start)].facebook_id.nunique() / d["contacts"][d["contacts"].creation_time <= scenario_start].facebook_id.nunique()

#Incoming Interactions
in_date_filter = (d["in"].datetime >= scenario_start) & (d["in"].datetime <= scenario_end)

active_user = d["in"][in_date_filter].facebook_id.nunique()
total_in = d["in"][in_date_filter].facebook_id.count()
in_per_user = round(total_in/active_user)

#Outgoing Interactions
out_date_filter = (d["out"].datetime >= scenario_start) & (d["out"].datetime <= scenario_end)

total_out = d["out"][out_date_filter].scenario_part_messages_count.sum()
out_per_user = round(total_out/active_user)

#Trackers
tracker_date_filter = (d["trackers"].datetime >= scenario_start) & (d["trackers"].datetime <= scenario_end)

total_trackers = d["trackers"][tracker_date_filter].facebook_id.count()
unique_trackers = d["trackers"][tracker_date_filter].facebook_id.nunique()

In [116]:
print("DELIVERABILITY")
print("- total direct messages: {}".format(messages_count))
print("- submitted: {}".format(submitted))
print("- delivered: {}".format(delivered))
print("- deliverability rate: {0:.2%}".format(deliverability_rate))
print("- failed: {}".format(failed))
print("")
print("READ")
print("- read: {}".format(read))
print("- read rate: {0:.2%}".format(read_rate))
print("")
print("CONTACTS")
print("- optins: {}".format(optins))
print("- new users: {}".format(new_users))
print("- retention rate: {0:.2%}".format(retention_rate))
print("- acquisition rate: {0:.2%}".format(acquisition_rate))
print("")
print("INCOMING")
print("- active users: {}".format(active_user))
print("- total incoming interactions (sent by user): {}".format(total_in))
print("- average incoming interactions per active users: {}".format(in_per_user))
print("")
print("OUTGOING")
print("- total outgoing interactions (sent to user): {}".format(total_out))
print("- average outgoing interactions per active users: {}".format(out_per_user))
print("")
print("TRACKERS")
print("- total clicks (on external links): {}".format(total_trackers))
print("- unique clickers (on external links): {}".format(unique_trackers))

DELIVERABILITY
- total direct messages: 5562
- submitted: 2908
- delivered: 2892
- deliverability rate: 99.45%
- failed: 2654

READ
- read: 2722
- read rate: 94.12%

CONTACTS
- optins: 3358
- new users: 2
- retention rate: 34.99%
- acquisition rate: 0.06%

INCOMING
- active users: 1149
- total incoming interactions (sent by user): 11689
- average incoming interactions per active users: 10.0

OUTGOING
- total outgoing interactions (sent to user): 25762
- average outgoing interactions per active users: 22.0

TRACKERS
- total clicks (on external links): 0
- unique clickers (on external links): 0
